# Chapter 4

## Creamos una vista temporal

In [0]:
%scala
val schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"

schema: String = date STRING, delay INT, distance INT, origin STRING, destination STRING

In [0]:
%scala
// In Scala
import org.apache.spark.sql.SparkSession 
val spark = SparkSession
 .builder
 .appName("SparkSQLExampleApp")
 .getOrCreate()
// Path to data set 
val csvFile= "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
// Read and create a temporary view
// Infer schema (note that for larger files you may want to specify the schema)
val df = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .load(csvFile)
// Create a temporary view
df.createOrReplaceTempView("us_delay_flights_tbl")

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5be5e8d7
csvFile: String = /databricks-datasets/learning-spark-v2/flights/departuredelays.csv
df: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]

In [0]:
%scala
df.columns

res2: Array[String] = Array(date, delay, distance, origin, destination)

## Consultas SQL

Buscamos todos los vuelos cuya distancia sea mayor a 1,000 millas

In [0]:
%scala
spark.sql("""SELECT distance, origin, destination 
FROM us_delay_flights_tbl WHERE distance > 1000 
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
distance|origin|destination|
+--------+------+-----------+
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
+--------+------+-----------+
only showing top 10 rows

Todos los vuelos entre San Francisco (SFO) y Chicago (ORD) con al menos dos horas de retraso

In [0]:
%scala
spark.sql("""SELECT date, delay, origin, destination 
FROM us_delay_flights_tbl 
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
 date|delay|origin|destination|
+-------+-----+------+-----------+
2190925| 1638| SFO| ORD|
1031755| 396| SFO| ORD|
1022330| 326| SFO| ORD|
1051205| 320| SFO| ORD|
1190925| 297| SFO| ORD|
2171115| 296| SFO| ORD|
1071040| 279| SFO| ORD|
1051550| 274| SFO| ORD|
3120730| 266| SFO| ORD|
1261104| 258| SFO| ORD|
+-------+-----+------+-----------+
only showing top 10 rows

Etiquetar todos los vuelos de EE. UU., independientemente de su origen y destino, con una indicación de los retrasos que experimentaron: retrasos muy largos (> 6 horas), retrasos largos (2 a 6 horas), etc. Agregaremos estas etiquetas legibles por humanos en una nueva columna llamada Flight_Delays

In [0]:
%scala
spark.sql("""SELECT delay, origin, destination,
 CASE
 WHEN delay > 360 THEN 'Very Long Delays'
 WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
 WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
 WHEN delay = 0 THEN 'No Delays'
 ELSE 'Early'
 END AS Flight_Delays
 FROM us_delay_flights_tbl
 ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
 333| ABE| ATL| Long Delays|
 305| ABE| ATL| Long Delays|
 275| ABE| ATL| Long Delays|
 257| ABE| ATL| Long Delays|
 247| ABE| DTW| Long Delays|
 247| ABE| ATL| Long Delays|
 219| ABE| ORD| Long Delays|
 211| ABE| ATL| Long Delays|
 197| ABE| DTW| Long Delays|
 192| ABE| ORD| Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows

**Ejercicio: Convertir la columna fecha a un formato legible**

In [0]:
%scala
import org.apache.spark.sql.functions._
val df_new = (df
         .withColumn("DateAndHour", to_timestamp(col("date"), "mm-dd hh:mm")))

import org.apache.spark.sql.functions._
df_new: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 4 more fields]

In [0]:
%scala
def toDateFormatUDF(dStr:String) : String  = {
  return s"${dStr(0)}${dStr(1)}${'-'}${dStr(2)}${dStr(3)}${' '}${dStr(4)}${dStr(5)}${':'}${dStr(6)}${dStr(7)}"
}

// test  it
toDateFormatUDF("02190925")

toDateFormatUDF: (dStr: String)String
res2: String = 02-19 09:25

In [0]:
%scala
df.selectExpr("toDateFormatUDF(date) as data_format").show(10, false)

at org.apache.spark.sql.errors.QueryCompilationErrors$.noSuchFunctionError(QueryCompilationErrors.scala:1794)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$LookupFunctions$$anonfun$apply$24.applyOrElse(Analyzer.scala:2750)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$LookupFunctions$$anonfun$apply$24.applyOrElse(Analyzer.scala:2736)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:492)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:94)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:492)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$3(TreeNode.scala:497)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren(TreeNode.scala:1143)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren$(TreeNode.scala:1142)
	at org.apache.spark.sql.catalyst.expressions.UnaryExpression.mapChildren(Expression.scala:552)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDownWithPruning(TreeNode.scala:497)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$transformExpressionsDownWithPruning$1(QueryPlan.scala:161)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$1(QueryPlan.scala:202)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:202)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.recursiveTransform$1(QueryPlan.scala:213)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$3(QueryPlan.scala:218)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.recursiveTransform$1(QueryPlan.scala:218)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$4(QueryPlan.scala:223)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:335)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:223)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsDownWithPruning(QueryPlan.scala:161)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsWithPruning(QueryPlan.scala:132)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveExpressionsWithPruning$1.applyOrElse(AnalysisHelper.scala:246)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveExpressionsWithPruning$1.applyOrElse(AnalysisHelper.scala:245)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$2(AnalysisHelper.scala:171)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:94)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$1(AnalysisHelper.scala:171)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:324)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning(AnalysisHelper.scala:169)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning$(AnalysisHelper.scala:165)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDownWithPruning(LogicalPlan.scala:30)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsWithPruning(AnalysisHelper.scala:99)
	at org.apa

In [0]:
%scala
spark.udf.register("toDateFormatUDF", toDateFormatUDF(_:String):String)

res61: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$9266/548813369@3cb1151a,StringType,List(Some(class[value[0]: string])),Some(class[value[0]: string]),Some(toDateFormatUDF),true,true)

In [0]:
%scala
val df_new = df.selectExpr("toDateFormatUDF(date) as DateAndHour")

df_new: org.apache.spark.sql.DataFrame = [DateAndHour: string]

In [0]:
%scala
df_new.show()

+--------+-----+--------+------+-----------+-----------+
 date|delay|distance|origin|destination|DateAndHour|
+--------+-----+--------+------+-----------+-----------+
01011245| 6| 602| ABE| ATL| null|
01020600| -8| 369| ABE| DTW| null|
01021245| -2| 602| ABE| ATL| null|
01020605| -4| 602| ABE| ATL| null|
01031245| -4| 602| ABE| ATL| null|
01030605| 0| 602| ABE| ATL| null|
01041243| 10| 602| ABE| ATL| null|
01040605| 28| 602| ABE| ATL| null|
01051245| 88| 602| ABE| ATL| null|
01050605| 9| 602| ABE| ATL| null|
01061215| -6| 602| ABE| ATL| null|
01061725| 69| 602| ABE| ATL| null|
01061230| 0| 369| ABE| DTW| null|
01060625| -3| 602| ABE| ATL| null|
01070600| 0| 369| ABE| DTW| null|
01071725| 0| 602| ABE| ATL| null|
01071230| 0| 369| ABE| DTW| null|
01070625| 0| 602| ABE| ATL| null|
01071219| 0| 569| ABE| ORD| null|
01080600| 0| 369| ABE| DTW| null|
+--------+-----+--------+------+-----------+-----------+
only showing top 20 rows

Las consultas anteriores se pueden expresar como consultas API. Por ejemplo la primera:

In [0]:
%scala
(df.select("distance", "origin", "destination")
 .where(col("distance") > 1000)
 .orderBy(desc("distance"))).show(10)

+--------+------+-----------+
distance|origin|destination|
+--------+------+-----------+
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
 4330| HNL| JFK|
+--------+------+-----------+
only showing top 10 rows

La segunda

In [0]:
%scala
//spark.sql("""SELECT date, delay, origin, destination 
//FROM us_delay_flights_tbl 
//WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
//ORDER by delay DESC""").show(10)

df
       .select("date", "delay", "origin", "destination")
       .where(col("delay") > 120 && col("origin") === "SFO" && col("destination") === "ORD").show()

+--------+-----+------+-----------+
 date|delay|origin|destination|
+--------+-----+------+-----------+
01011410| 124| SFO| ORD|
01022330| 326| SFO| ORD|
01021410| 190| SFO| ORD|
01101410| 184| SFO| ORD|
01190925| 297| SFO| ORD|
01241110| 139| SFO| ORD|
01301800| 167| SFO| ORD|
01011237| 122| SFO| ORD|
01032258| 163| SFO| ORD|
01031920| 193| SFO| ORD|
01031755| 396| SFO| ORD|
01071040| 279| SFO| ORD|
01161210| 225| SFO| ORD|
01221040| 215| SFO| ORD|
01261104| 258| SFO| ORD|
01020720| 145| SFO| ORD|
01021205| 154| SFO| ORD|
01031550| 131| SFO| ORD|
01041205| 126| SFO| ORD|
01051205| 320| SFO| ORD|
+--------+-----+------+-----------+
only showing top 20 rows

La tercera

In [0]:
%scala
//spark.sql("""SELECT delay, origin, destination,
//CASE
//WHEN delay > 360 THEN 'Very Long Delays'
//WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
//WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
//WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
//WHEN delay = 0 THEN 'No Delays'
//ELSE 'Early'
//END AS Flight_Delays
//FROM us_delay_flights_tbl
//ORDER BY origin, delay DESC""").show(10)


val df_3 = df.withColumn("Flight_Delays",
                        when(col("delay") > 360,
                            lit("Very Long Delays"))
                        when(col("delay") > 120 && col("delay") < 360,
                            lit("Long Delays"))
                        when(col("delay") > 60 && col("delay") < 120,
                            lit("Short Delays"))
                        when(col("delay") > 0 && col("delay") < 60,
                            lit("Tolerable Delays"))
                        when(col("delay") === 0,
                            lit("No Delays"))
                        otherwise(lit("Early")))
.orderBy(desc("origin"))
.show(10)

+--------+-----+--------+------+-----------+----------------+
 date|delay|distance|origin|destination| Flight_Delays|
+--------+-----+--------+------+-----------+----------------+
03010941| -2| 139| YUM| PHX| Early|
03021130| 9| 139| YUM| PHX|Tolerable Delays|
03011255| -9| 139| YUM| PHX| Early|
03011130| -7| 139| YUM| PHX| Early|
03010800| -6| 139| YUM| PHX| Early|
03010600| 5| 206| YUM| LAX|Tolerable Delays|
03011530| -6| 206| YUM| LAX| Early|
03020941| -10| 139| YUM| PHX| Early|
03021255| -5| 139| YUM| PHX| Early|
03021859| -10| 139| YUM| PHX| Early|
+--------+-----+--------+------+-----------+----------------+
only showing top 10 rows

df_3: Unit = ()

## Creando Tablas y Databases de SQL

### Databases

In [0]:
%scala
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

res17: org.apache.spark.sql.DataFrame = []

Ahora todas las tablas que se creen estarán dentro de esa base de datos

d
### Tables

#### Managed
Las tablas administradas administran tanto los metadatos como los datos. Si se ejecuta un DROP se eliminarían los datos reales y los metadatos.

In [0]:
%scala
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT, distance INT, origin STRING, destination STRING)").show()

++
|
++
++

Aunque cierre un cluster y abra otro, lo que he hecho en sesiones anteriores se guarda en el fichero de metadatos. Al abrir otra sesión, la tabla no aparece en el contenido de las tablas, pero no me deja volverla a crear porque ya existen los metadatos de la misma. Tendré que eliminar el registro

In [0]:
%scala
dbutils.fs.rm("dbfs:/user/hive/warehouse/learn_spark_db.db/managed_us_delay_flights_tbl",recurse=true)

res20: Boolean = true

#### Unmanaged
En las tablas no administradas, Spark solo administra los metadatos mientras que el propio usuario maneja los datos. Aquí el DROP solo eliminaría los metadatos.

In [0]:
%scala
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, 
 distance INT, origin STRING, destination STRING) 
 USING csv OPTIONS (PATH 
 '/databricks-datasets/learning-spark-v2/flights/departuredelays.csv')""")

res22: org.apache.spark.sql.DataFrame = []

### Creando vistas
Las vistas se crean sobre tablas existentes y pueden ser globales o session-scoped. Las globales son visibles en todas las Spark-Sessions de un cluster y la segunda solo es visible para una sola Spark-session (son temporales, desaparecen después de la finalización de la Spark Application.

#### Global
Son visibles en todos los SparkSession en un cluster dado

In [0]:
%scala
//A partir de una tabla existente: SQL
spark.sql("""CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
 SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
 origin = 'SFO'""")

res24: org.apache.spark.sql.DataFrame = []

In [0]:
%scala
// API
val df_sfo = spark.sql("""SELECT date, delay, origin, destination FROM 
 us_delay_flights_tbl WHERE origin = 'SFO'""")
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_sfo.show(5)

+-------+-----+------+-----------+
 date|delay|origin|destination|
+-------+-----+------+-----------+
1011250| 55| SFO| JFK|
1012230| 0| SFO| JFK|
1010705| -7| SFO| JFK|
1010620| -3| SFO| MIA|
1010915| -3| SFO| LAX|
+-------+-----+------+-----------+
only showing top 5 rows

df_sfo: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 2 more fields]

In [0]:
%scala
spark.sql("""SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view""").show()

+-------+-----+------+-----------+
 date|delay|origin|destination|
+-------+-----+------+-----------+
1011250| 55| SFO| JFK|
1012230| 0| SFO| JFK|
1010705| -7| SFO| JFK|
1010620| -3| SFO| MIA|
1010915| -3| SFO| LAX|
1011005| -8| SFO| DFW|
1011800| 0| SFO| ORD|
1011740| -7| SFO| LAX|
1012015| -7| SFO| LAX|
1012110| -1| SFO| MIA|
1011610| 134| SFO| DFW|
1011240| -6| SFO| MIA|
1010755| -3| SFO| DFW|
1010020| 0| SFO| DFW|
1010705| -6| SFO| LAX|
1010925| -3| SFO| ORD|
1010555| -6| SFO| ORD|
1011105| -8| SFO| DFW|
1012330| 32| SFO| ORD|
1011330| 3| SFO| DFW|
+-------+-----+------+-----------+
only showing top 20 rows

**Eliminar vistas**

In [0]:
%scala
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")

res67: Boolean = true

#### Temporal
Las vistas son visibles para un único SparkSessios y se eliminan después de finalizar la aplicación Spark

In [0]:
%scala
spark.sql("""CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS
 SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE
 origin = 'JFK'""")

res33: org.apache.spark.sql.DataFrame = []

In [0]:
%scala
//API
val df_jfk = spark.sql("""SELECT date, delay, origin, destination FROM 
 us_delay_flights_tbl WHERE origin = 'JFK'""")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")
df_jfk.show(5)

+-------+-----+------+-----------+
 date|delay|origin|destination|
+-------+-----+------+-----------+
1010900| 14| JFK| LAX|
1011200| -3| JFK| LAX|
1011900| 2| JFK| LAX|
1011700| 11| JFK| LAS|
1010800| -1| JFK| SFO|
+-------+-----+------+-----------+
only showing top 5 rows

df_jfk: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 2 more fields]

In [0]:
%scala
spark.sql("""SELECT * FROM us_origin_airport_JFK_tmp_view""").show()

+-------+-----+------+-----------+
 date|delay|origin|destination|
+-------+-----+------+-----------+
1010900| 14| JFK| LAX|
1011200| -3| JFK| LAX|
1011900| 2| JFK| LAX|
1011700| 11| JFK| LAS|
1010800| -1| JFK| SFO|
1011540| -4| JFK| DFW|
1011705| 5| JFK| SAN|
1011530| -3| JFK| SFO|
1011630| -3| JFK| SJU|
1011345| 2| JFK| LAX|
1011545| -3| JFK| LAX|
1011510| -1| JFK| MIA|
1011745| 7| JFK| SFO|
1011250| 3| JFK| BOS|
1011645| 142| JFK| LAX|
1012135| -2| JFK| LAX|
1011715| 18| JFK| ORD|
1011615| 25| JFK| IAH|
1011850| -2| JFK| SEA|
1011725| -5| JFK| BOS|
+-------+-----+------+-----------+
only showing top 20 rows

**Eliminar vistas**

In [0]:
%scala
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

res68: Boolean = true

## Metadatos
Spark administra los metadatos asociados a cada tabla, ya sea administrada o no administrada. Para administrar los metadatos se utiliza Catalog la cual es una herramienta de alto nivel de Spark SQL para almacenar metadatos.

Después de crear la vartiable de la SparkSession, se puede acceder al metadata almacenado de la siaguiente manera:

In [0]:
%scala
display(spark.catalog.listTables(dbName="global_temp"))

name,database,description,tableType,isTemporary
us_origin_airport_sfo_global_tmp_view,global_temp,null,TEMPORARY,true
us_delay_flights_tbl,null,null,TEMPORARY,true
us_origin_airport_jfk_tmp_view,null,null,TEMPORARY,true


**Almacenamiento en caché de tablas SQL**

Se puede cache y uncahe tablas SQL y vistas. Además, si especificas la tabla como LAZY, se guardará en caché cuando se utiliza la tabla por primera vez y no inmediatamente cuando se crea.

In [0]:
%scala
//In SQL
spark.sql("CACHE [LAZY] TABLE <table-name>")
spark.sql("UNCACHE TABLE <table-name>")

## Leyendo tablas a DataFrames
Si ya hay una base de datos learn_spark_db y una tabla us_delay_flights_tbl preparadas para ser utilizadas. En vez de importar datos directamente del JSON file para tener un dataframe, podemos simplemente ejecutar una consulta SQL a la tabla y asignarle como resultado un DataFrame

In [0]:
%scala
//Dos formas
val usFlightsDF = spark.sql("SELECT * FROM us_delay_flights_tbl")
val usFlightsDF2 = spark.table("us_delay_flights_tbl")
usFlightsDF.show(5)

usFlightsDF: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]
usFlightsDF2: org.apache.spark.sql.DataFrame = [date: string, delay: int ... 3 more fields]

## Data Sources for DataFrames and SQL Tables
Spark SQL proporciona una gran variedad de data sources. Además de proporcionar un conjunto de métodos reales para leer y escribir desde estas fuentes utilizando la Data Sources API.

### DataFrameReader
Es la herramienta con la que se lee una fuente de datos en un DataFrame. Tiene un formato definido y un patrón recomendado de uso.

```DataFrameReader.format(args).option("key", "value").schema(args).load()```

**Opciones**

| Method | Arguments | Description |
| --- | --- | --- |
| format() | "parquet", "csv", "txt", "json","jdbc", "orc", "avro", etc. | If you don’t specify this method, then the default ismParquet or whatever is set in spark.sql.sources.default.|
| option() |("mode", {PERMISSIVE / FAILFAST / DROPMALFORMED } )("inferSchema", {true / false}) ("path", "path_file_data_source") | A series of key/value pairs and options. The Spark documentation shows some examples and explains the different modes and their actions. The default mode is PERMISSIVE. The "inferSchema" and "mode" options are specific to the JSON and CSV file formats.|
| schema() | DDL String or StructType, e.g., 'A INT, B STRING' orStructType(...) | For JSON or CSV format, you can specify to infer the schema in the option() method. Generally, providing a schema for any format makes loading faster and ensures your data conforms to the expected schema.|
| load() | "/path/to/data/source" | The path to the data source. This can be empty if specified in option("path", "...").|

**Ejemplos**

In [0]:
%scala
// Use Parquet 
val file = """/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet"""
val df = spark.read.format("parquet").load(file)
// Use Parquet; you can omit format("parquet") if you wish as it's the default
//val df2 = spark.read.load(file)
// Use CSV
val df3 = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .option("mode", "PERMISSIVE")
 .load("/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*")
// Use JSON
val df4 = spark.read.format("json")
 .load("/databricks-datasets/learning-spark-v2/flights/summary-data/json/*")

df.show(5)
df3.show(5)
df4.show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 1|
 United States| Ireland| 264|
 United States| India| 69|
 Egypt| United States| 24|
Equatorial Guinea| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 1|
 United States| Ireland| 264|
 United States| India| 69|
 Egypt| United States| 24|
Equatorial Guinea| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 15|
 United States| Croatia| 1|
 United States| Ireland| 344|
 Egypt| United States| 15|
 United States| India| 62|
+-----------------+-------------------+-----+
only showing top 5 rows

file: String = /databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
df3: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
df4: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

En general, no se necesita ningún esquema cuando se lee desde una fuente de datos estática de Parquet; los metadatos de Parquet generalmente contienen el esquema,  por lo que se deduce.

### DataFrameWriter
Guarda o escribe datos en una fuente de datos integrada especificada y estos son sus patrones de uso:
```
DataFrameWriter.format(args)
 .option(args)
 .bucketBy(args)
 .partitionBy(args)
 .save(path)
```
o

```
DataFrameWriter.format(args).option(args).sortBy(args).saveAsTable(table)
```

**Opciones**

| Method | Arguments | Description |
| --- | --- | --- |
| format() | "parquet", "csv", "txt", "json","jdbc", "orc", "avro", etc. | If you don’t specify this method, then the default is Parquet or whatever is set in spark.sql.sources.default.|
| option() | ("mode", {append / overwrite / ignore / error or errorifexists} ) ("mode", {SaveMode.Overwrite / SaveMode.Append, Save Mode.Ignore, SaveMode.ErrorIfExists}) ("path", "path_to_write_to") | A series of key/value pairs and options. The Spark documentation shows some examples. This is an overloaded method. The default mode options are error or error ifexists and SaveMode.ErrorIfExists; they throw an exception at runtime if the data already exists.|
| bucketBy() | (numBuckets, col, col..., coln) | The number of buckets and names of columns to bucket by. Uses Hive’s bucketing scheme on a filesystem.|
| save() | "/path/to/data/source" | The path to save to. This can be empty if specified in option("path", "..."). |
| saveAsTable() | "table_name" | The table to save to.|

**Ejemplo**

In [0]:
%scala
//Use JSON
//El nombre me lo pone Databricks. La location es el directorio en el que se guarda el fichero. Si le quier camciar el nombre tengo que hacer un rename
val location = """/learning-spark-v2/sf-fire/Ejemplo_save_json"""
df.write.format("json").mode("overwrite").save(location)

location: String = /learning-spark-v2/sf-fire/Ejemplo_save_json

## Fuentes de datos

### Parquet
La fuente de datos predeterminada de Spark

#### Leer de archivos Parquet en un DataFrame

In [0]:
%scala
val file = """/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/"""
val df = spark.read.format("parquet").load(file)
df.show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 1|
 United States| Ireland| 264|
 United States| India| 69|
 Egypt| United States| 24|
Equatorial Guinea| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

file: String = /databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

#### Leer de archivos Parquet en una tabla SQL

In [0]:
%scala
//Creamos la base de datos
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING parquet
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/" )
""")

res19: org.apache.spark.sql.DataFrame = []

In [0]:
%scala
//Leemos la tabla
spark.sql("SELECT * FROM us_delay_flights_tbl").show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 1|
 United States| Ireland| 264|
 United States| India| 69|
 Egypt| United States| 24|
Equatorial Guinea| United States| 1|
+-----------------+-------------------+-----+
only showing top 5 rows

#### Escribir DataFrames a archivos Parquet

In [0]:
%scala
df.write.format("parquet")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/parquet/df_parquet")

#### Escribir DataFrames a tablas SQL

In [0]:
%scala
df.write
 .mode("overwrite")
 .saveAsTable("us_delay_flights_tbl")

### JSON
En Spark se soporta tanto el formato sigle-line mode como el multiline mode.

#### Leer de archivos JSON en un DataFrame

In [0]:
%scala
val file = "/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
val df = spark.read.format("json").load(file)


#### Leer de archivos JSON en una tabla SQL

In [0]:
%scala
// Creamos la base de datos
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING json
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
 )""")

In [0]:
%scala
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

d
#### Escribir DataFrames a archivos JSON

In [0]:
%scala
df.write.format("json")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/json/df_json")

**Opciones**

| Property name | Values | Meaning | Scope
| --- | --- | --- | --- |
| compression | none, uncompressed, bzip2, deflate, gzip, lz4, or snappy | Use this compression codec for writing. Note that read will only detect the compression or codec from the file extension.| Write |
| dateFormat | yyyy-MM-dd or DateTimeFormatter | Use this format or any format from Java’s DateTime Formatter. | Read/ write |
| multiLine | true, false | Use multiline mode. Default is false (single-line mode). | Read
| allowUnquoted FieldNames | true, false | Allow unquoted JSON field names. Default is false. | Read

### CSV

#### Leer de archivos CSV en un DataFrame

In [0]:
%scala
val file = "/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
val schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"
val df = spark.read.format("csv")
 .schema(schema)
 .option("header", "true")
 .option("mode", "FAILFAST") // Exit if any errors
 .option("nullValue", "") // Replace any null data with quotes
 .load(file)

#### Leer de archivos CSV en una tabla SQL

In [0]:
%scala
//Creamos la base de datos
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING csv
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*",
 header "true",
 inferSchema "true",
 mode "FAILFAST"
 )""")

In [0]:
%scala
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)


#### Escribir DataFrames a archivos CSV

In [0]:
%scala
df.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_csv")

**Opciones**
## TABLA
Página 128

### AVRO

#### Leer de archivos AVRO en un DataFrame

In [0]:
%scala
val df = spark.read.format("avro")
.load("/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*")
df.show(false)

#### Leer de archivos AVRO en una tabla SQL

In [0]:
%scala
//Creamos la base de datos
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW episode_tbl
 USING avro
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*"
 )""")


In [0]:
%scala
spark.sql("SELECT * FROM episode_tbl").show(false)

#### Escribir DataFrames a archivos AVRO

In [0]:
%scala
df.write
 .format("avro")
 .mode("overwrite")
 .save("/tmp/data/avro/df_avro")

**Opciones**

##TABLA
Página 130

### ORC

#### Leer de archivos ORC en un DataFrame

In [0]:
%scala
val file = "/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
val df = spark.read.format("orc").load(file)
df.show(10, false)

#### Leer de archivos ORC en una tabla SQL

In [0]:
%scala
//Creo la base de datos 
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
 USING orc
 OPTIONS (
 path "/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
 )""")

In [0]:
%scala
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

#### Escribir DataFrames a archivos ORC

In [0]:
%scala
// In Scala
df.write.format("orc")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/orc/df_orc")


### Imágenes

#### Reading an image file into a DataFrame

In [0]:
%scala
import org.apache.spark.ml.source.image
val imageDir = "/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
val imagesDF = spark.read.format("image").load(imageDir)
imagesDF.printSchema
imagesDF.select("image.height", "image.width", "image.nChannels", "image.mode",
 "label").show(5, false)

### Binary Files

#### Reading a binary file into a DataFrame
El siguiente código lee todos los archivos JPG del directorio de entrada con cualquier directorio particionado

In [0]:
%scala
val path = "/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
val binaryFilesDF = spark.read.format("binaryFile")
 .option("pathGlobFilter", "*.jpg")
 .load(path)
binaryFilesDF.show(5)

Para ignorar la detección de datos de partición en un directorio, puede configurar ```recursiveFile Lookup to "true"```:

In [0]:
%scala
val binaryFilesDF = spark.read.format("binaryFile")
 .option("pathGlobFilter", "*.jpg")
 .option("recursiveFileLookup", "true")
 .load(path)
binaryFilesDF.show(5)

# APUNTES
Con \%fs puedo ejecutar comandos linux en el Notebook de Databricks. Se puede hacer un ls, ver directorios, crearlos y eliminarlos. En la siguiente web ```//https://docs.databricks.com/_static/notebooks/dbutils.html``` se observan todos los comandos disponibles.

**Ejemplos**

In [0]:
%fs
ls

path,name,size,modificationTime
dbfs:/Ejemplo_save_json.json/,Ejemplo_save_json.json/,0,0
dbfs:/FileStore/,FileStore/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/learning-spark-v2/,learning-spark-v2/,0,0
dbfs:/tmp/,tmp/,0,0
dbfs:/user/,user/,0,0


In [0]:
%fs
ls /learning-spark-v2/sf-fire/Ejemplo_save_json

path,name,size,modificationTime
dbfs:/learning-spark-v2/sf-fire/Ejemplo_save_json/_SUCCESS,_SUCCESS,0,1651142577000
dbfs:/learning-spark-v2/sf-fire/Ejemplo_save_json/_committed_2238006556770501276,_committed_2238006556770501276,114,1651142577000
dbfs:/learning-spark-v2/sf-fire/Ejemplo_save_json/_started_2238006556770501276,_started_2238006556770501276,0,1651142576000
dbfs:/learning-spark-v2/sf-fire/Ejemplo_save_json/part-00000-tid-2238006556770501276-4704d525-889b-423b-8d58-a96e6976647e-280-1-c000.json,part-00000-tid-2238006556770501276-4704d525-889b-423b-8d58-a96e6976647e-280-1-c000.json,21353,1651142576000


In [0]:
%fs
rm -r '/"'

res38: Boolean = false